# Notes - Xarray

## Terminology

- DataArray: A multi-dimensional array with labeled or named dimensions. DataArray objects add metadata such as dimension names, coordinates, and attributes. For example, an array is var(time, level, lat, lon).
- DataSet: A dict-like collection of DataArray objects with aligned dimensions. For example, a dataset contains temperature(time, level, lat, lon) and precipitation(time, lat, lon).

## References

- Unidata Xarray Introduction, https://unidata.github.io/python-training/workshop/XArray/xarray-introduction/
- Xarray quick overview, https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html
- Xarray computation, https://docs.xarray.dev/en/stable/user-guide/computation.html


In [2]:
import numpy as np
import xarray as xr
import io, os, sys, types
import yhc_module as yhc

## Create and Modify a DataArray

### Create a DataArray

Xarray - https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html#create-a-dataarray

Unidata - https://unidata.github.io/python-training/workshop/XArray/xarray-introduction/#DataArray

In [17]:
#--- Create some sample "temperature" data
data = 283 + 5 * np.random.randn(5, 3, 4)

time = np.arange(0,5)
lat = np.linspace(-120., 60., 3)
lon = np.linspace(25.,55.,4)

#--- create a DataArray & set attributes
temp = xr.DataArray(data, dims=['time', 'lat', 'lon'], coords=[time, lat, lon])

temp.attrs['units'] = "K"
temp.attrs['long_name'] = "Temperature"

with xr.set_options(keep_attrs=True):  # keep attributes after operation
  temp_degC = temp - 273.15
temp_degC.attrs['units'] = "C"
yhc.printv(temp_degC, "temp_degC")

#--- put all in one line
arr = xr.DataArray(np.random.RandomState(0).randn(2, 3), [("x", ["a", "b"]), ("y", [10, 20, 30])])
yhc.printv(arr, "arr")

--------------
temp_degC
<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[ 9.46728032, 12.22009414, 13.83345992,  6.96229321],
        [14.11250087, 13.24816373,  1.87497189, 12.33044252],
        [15.01317053, 13.9207479 ,  5.04936473,  2.15194986]],

       [[12.84860934, 12.76793515,  9.81908196, 14.82508736],
        [ 5.02295605, 16.95145695, 14.47079308,  6.5745184 ],
        [-1.68205605, -7.46520115, 11.77579848, -0.23886744]],

       [[ 8.3249198 , 12.75556155, 11.47552588,  3.86024961],
        [12.44596445,  8.27606915, 11.73183796,  9.7339703 ],
        [ 9.87113793, 12.33100254, 10.72159994,  6.54408809]],

       [[ 5.28556346, 10.67149369, 13.55035168, 11.60503475],
        [ 5.227882  , 10.42462381,  9.29458744, 13.39606813],
        [15.92646692, 12.50526708,  0.54748382,  7.81053211]],

       [[ 2.64881976,  4.64966349, 16.61786489,  9.22661074],
        [16.25891119,  5.33451418,  7.58168571, -0.11997918],
        [14.99909718, 10.38321093, 14.02776244,  4.636

### Copy from an exisitng DataArray

In [41]:
#--- create a sample array
arr0 = xr.DataArray(np.random.RandomState(0).randn(2, 3), [("x", ["a", "b"]), ("y", [10, 20, 30])])
yhc.printv(arr0,'original arr0')

#--- arr1 = arr0, I think this means arr1 and arr0 share the same memory. 
#    So if you change arr1, arr0 is automatically changed as well. 
arr1 = arr0
arr1[0,0] = 100.
arr1[1,1] = 50.
arr2 = arr1 - arr0      # arr1 still equal to arr0, even though I have already change the values...
yhc.printv(arr1,'arr0')
yhc.printv(arr1,'arr1')
yhc.printv(arr2,'arr2')

#--- use copy() to make a copy of an existing DataArray.
arrA = arr0.copy()
arrA[0,0] = 75.
arrA[1,1] = 30.
arrB = arrA - arr0
yhc.printv(arr0,'arr0')
yhc.printv(arrA,'arrA')
yhc.printv(arrB,'arrB')

#--- copy an existing dataArray and reset all elements to zeros
arrC = arr0.copy().where(arr0 == 0., 0., 0.)
# Cannot use arrC = 0., This will return arrC = 0.
yhc.printv(arrC,'arrC')

--------------
original arr0
<xarray.DataArray (x: 2, y: 3)>
array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30

--------------
arr0
<xarray.DataArray (x: 2, y: 3)>
array([[100.        ,   0.40015721,   0.97873798],
       [  2.2408932 ,  50.        ,  -0.97727788]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30

--------------
arr1
<xarray.DataArray (x: 2, y: 3)>
array([[100.        ,   0.40015721,   0.97873798],
       [  2.2408932 ,  50.        ,  -0.97727788]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30

--------------
arr2
<xarray.DataArray (x: 2, y: 3)>
array([[0., 0., 0.],
       [0., 0., 0.]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30

--------------
arr0
<xarray.DataArray (x: 2, y: 3)>
array([[100.        ,   0.40015721,   0.97873798],
       [  2.2408932 ,  50.        

### Modify coordinates of a DataArray

Xarray, modify coordinates
https://docs.xarray.dev/en/stable/generated/xarray.Dataset.assign_coords.html

In [3]:
#--- create a sample
arr = xr.DataArray(np.random.RandomState(0).randn(2, 3), [("x", ["a", "b"]), ("y", [10, 20, 30])])
yhc.printv(arr, "original")

#--- print out coordinate
arr.coords["x"]   # print out coordinate "x" values

#--- assign completely new coordinate values
x_new = ["cc","dd"]
arr = arr.assign_coords({"x":x_new}) # modidy the arraymodify "x" coordinate from [a,b] to [c,d]
    #arr.assign_coords({"x":x_new})    # This will show new coordinate but not modidy the array
yhc.printv(arr,"modify x")

#--- arthemetric operation of exisitng coordiate values
arr.assign_coords(y=arr.y+1)   # modify the coordinate values
arr = arr.assign_coords(y=((arr.y+5)/2)+1.)
yhc.printv(arr,"modify y")

#--- put all in one line
arr = arr.assign_coords({"x":["ee","ff"], "y":arr.y+1})
yhc.printv(arr, "modify x and y")

#--- change coordinate names
arr = arr.rename({"x":"x2", "y":"y2"})  # rename coordinate
yhc.printv(arr, 'rename')

#--- New coordinate can also be attached to an existing dimension
y_new = [11,12,13]
arr = arr.assign_coords(y_new=("y2", y_new))
yhc.printv(arr, 'add y_new as a coordinate variable')

#--- Add a new coordinate and the original array would be expanded 
z = np.array([1,2,3,4])
arr = arr.expand_dims(z=z)   # add a new coordinate "z". The "z" values are np.arrry *z*.
yhc.printv(arr, "add a new z coordiate")


--------------
original
<xarray.DataArray (x: 2, y: 3)>
array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30

--------------
modify x
<xarray.DataArray (x: 2, y: 3)>
array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])
Coordinates:
  * x        (x) <U2 'cc' 'dd'
  * y        (y) int64 10 20 30

--------------
modify y
<xarray.DataArray (x: 2, y: 3)>
array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])
Coordinates:
  * x        (x) <U2 'cc' 'dd'
  * y        (y) float64 8.5 13.5 18.5

--------------
modify x and y
<xarray.DataArray (x: 2, y: 3)>
array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])
Coordinates:
  * x        (x) <U2 'ee' 'ff'
  * y        (y) float64 9.5 14.5 19.5

--------------
rename
<xarray.DataArray (x2: 2, y2: 3)>
array(

## Selection

Unidata - https://unidata.github.io/python-training/workshop/XArray/xarray-introduction/#Selection

### Selection Method 1: use indexing

In [4]:
#--- Method 1: use indexing
var = temp[0, 1:2, :]
var

<xarray.DataArray (lat: 1, lon: 4)>
array([[275.25086233, 291.31080937, 275.7695059 , 286.89454691]])
Coordinates:
    time     int64 0
  * lat      (lat) float64 -30.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_name:  Temperature

### Selection Method 2: Use name dimension & slicing

In [5]:
#--- Method 2; use name dimension
temp.coords  # check out variable dimension
print(temp.coords)

#--- select specific values in coordinates
var = temp.sel(time=1, lat=-30., lon=25)
var = temp.sel(time=1, lat=-30.2, lon=25.3, method='nearest')

print('------------')
print(var)

var = temp.sel(time=1, lon=25)
print('------------')
print(var)

#--- Slicing with Selection
var = temp.sel(time=slice(0,2), lat=-30., lon=slice(-1000.,1000.))
print('------------')
print(var)

Coordinates:
  * time     (time) int64 0 1 2 3 4
  * lat      (lat) float64 -120.0 -30.0 60.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
------------
<xarray.DataArray ()>
array(287.20443164)
Coordinates:
    time     int64 1
    lat      float64 -30.0
    lon      float64 25.0
Attributes:
    units:      K
    long_name:  Temperature
------------
<xarray.DataArray (lat: 3)>
array([275.72880053, 287.20443164, 284.12549956])
Coordinates:
    time     int64 1
  * lat      (lat) float64 -120.0 -30.0 60.0
    lon      float64 25.0
Attributes:
    units:      K
    long_name:  Temperature
------------
<xarray.DataArray (time: 3, lon: 4)>
array([[275.25086233, 291.31080937, 275.7695059 , 286.89454691],
       [287.20443164, 282.53698475, 282.3709815 , 273.96060973],
       [276.99008459, 286.07485234, 290.43792342, 284.88393184]])
Coordinates:
  * time     (time) int64 0 1 2
    lat      float64 -30.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_n

### Selection Method 3: use .loc

In [6]:
#*** Useful if already knowing the range to each coordinate

# temp is temp(time, lat, lon)
var = temp.loc[0:4, -120:30, :]
print(var)

<xarray.DataArray (time: 5, lat: 2, lon: 4)>
array([[[278.6057103 , 285.04751953, 277.7746131 , 276.7385358 ],
        [275.25086233, 291.31080937, 275.7695059 , 286.89454691]],

       [[275.72880053, 281.43300473, 289.90382089, 281.28954146],
        [287.20443164, 282.53698475, 282.3709815 , 273.96060973]],

       [[287.84266885, 280.75106687, 279.53445397, 285.92322316],
        [276.99008459, 286.07485234, 290.43792342, 284.88393184]],

       [[292.69957208, 287.20869206, 284.59255505, 281.08802581],
        [287.32836227, 286.38091435, 272.74650431, 288.09922923]],

       [[277.60186545, 279.10669745, 282.07485037, 286.8258722 ],
        [280.2016732 , 289.47399732, 284.13505281, 286.26634133]]])
Coordinates:
  * time     (time) int64 0 1 2 3 4
  * lat      (lat) float64 -120.0 -30.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_name:  Temperature


### Selection Method 4: where() to conditionally switch between values

In [7]:
#--- use where()
var = temp.where(temp > 280.)  # if temp < 280., it would become nan
var

#--- create a mask, but fail.
#xr.where(temp > 280., "positive", "negative")

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[         nan, 285.04751953,          nan,          nan],
        [         nan, 291.31080937,          nan, 286.89454691],
        [         nan, 285.17922119,          nan, 287.20638945]],

       [[         nan, 281.43300473, 289.90382089, 281.28954146],
        [287.20443164, 282.53698475, 282.3709815 ,          nan],
        [284.12549956,          nan, 282.8242116 ,          nan]],

       [[287.84266885, 280.75106687,          nan, 285.92322316],
        [         nan, 286.07485234, 290.43792342, 284.88393184],
        [292.19964036,          nan, 282.39420138, 281.73048691]],

       [[292.69957208, 287.20869206, 284.59255505, 281.08802581],
        [287.32836227, 286.38091435,          nan, 288.09922923],
        [285.11570046, 292.11550521,          nan, 282.18909982]],

       [[         nan,          nan, 282.07485037, 286.8258722 ],
        [280.2016732 , 289.47399732, 284.13505281, 286.26634133],
        [285.56485241, 285.80664221,          nan,          nan]]])
Coordinates:
  * time     (time) int64 0 1 2 3 4
  * lat      (lat) float64 -120.0 -30.0 60.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_name:  Temperature

## Reshaping and reorganizing data

### Reordering dimensions, Expand/Squeeze data
https://docs.xarray.dev/en/stable/user-guide/reshaping.html#reordering-dimensions

In [8]:
#--- create a sample
ds = xr.Dataset({"foo": (("x", "y", "z"), [[[42]]]), "bar": (("y", "z"), [[24]])})
yhc.printv(ds,'original')

#--- reorder dimensions of all variables
ds1 = ds.transpose("y", "z", "x")
    #ds.transpose()          # reverses all dimensions
    #ds.transpose(..., "x")  # move "x" to the last dimension
yhc.printv(ds1,'reorder dims')

#--- expand data
ds_expand = ds.expand_dims(w=np.zeros(2))
yhc.printv(ds_expand, "expand dims")

#--- squeeze data
ds_squeeze = ds.squeeze("x")
yhc.printv(ds_squeeze, "squeeze dims")

--------------
original
<xarray.Dataset>
Dimensions:  (x: 1, y: 1, z: 1)
Dimensions without coordinates: x, y, z
Data variables:
    foo      (x, y, z) int64 42
    bar      (y, z) int64 24

--------------
reorder dims
<xarray.Dataset>
Dimensions:  (x: 1, y: 1, z: 1)
Dimensions without coordinates: x, y, z
Data variables:
    foo      (y, z, x) int64 42
    bar      (y, z) int64 24

--------------
expand dims
<xarray.Dataset>
Dimensions:  (x: 1, y: 1, z: 1, w: 2)
Coordinates:
  * w        (w) float64 0.0 0.0
Dimensions without coordinates: x, y, z
Data variables:
    foo      (w, x, y, z) int64 42 42
    bar      (w, y, z) int64 24 24

--------------
squeeze dims
<xarray.Dataset>
Dimensions:  (y: 1, z: 1)
Dimensions without coordinates: y, z
Data variables:
    foo      (y, z) int64 42
    bar      (y, z) int64 24



### Converting between datasets and arrays
https://docs.xarray.dev/en/stable/user-guide/reshaping.html#converting-between-datasets-and-arrays


In [9]:
#--- convert between a Dataset and a DataArray. All variables will be on a new dimension, variable
arr = ds.to_array()
yhc.printv(ds, 'Dataset')
yhc.printv(arr, 'DataArray')

ds1 = arr.to_dataset(dim="variable")
yhc.printv(ds1, 'Back to Dataset')


--------------
Dataset
<xarray.Dataset>
Dimensions:  (x: 1, y: 1, z: 1)
Dimensions without coordinates: x, y, z
Data variables:
    foo      (x, y, z) int64 42
    bar      (y, z) int64 24

--------------
DataArray
<xarray.DataArray (variable: 2, x: 1, y: 1, z: 1)>
array([[[[42]]],


       [[[24]]]])
Coordinates:
  * variable  (variable) <U3 'foo' 'bar'
Dimensions without coordinates: x, y, z

--------------
Back to Dataset
<xarray.Dataset>
Dimensions:  (x: 1, y: 1, z: 1)
Dimensions without coordinates: x, y, z
Data variables:
    foo      (x, y, z) int64 42
    bar      (x, y, z) int64 24



### Stack and unstack
https://docs.xarray.dev/en/stable/user-guide/reshaping.html#stack-and-unstack

In [10]:
#--- create a sample DataArray
arr = xr.DataArray(np.random.randn(2, 3), coords=[("x", ["a", "b"]), ("y", [0, 1, 2])])

#--- stack the array
arr_z = arr.stack(z=("x","y"))

#--- unstack the array
arr_unstack = arr_z.unstack("z")

yhc.printv(arr, 'original')
yhc.printv(arr_z, 'merge x,y dims to z')
yhc.printv(arr_unstack, 'unstack z')

--------------
original
<xarray.DataArray (x: 2, y: 3)>
array([[ 0.42387457,  1.66837469,  1.00810077],
       [-0.9651798 ,  1.67118143, -0.84389286]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 0 1 2

--------------
merge x,y dims to z
<xarray.DataArray (z: 6)>
array([ 0.42387457,  1.66837469,  1.00810077, -0.9651798 ,  1.67118143,
       -0.84389286])
Coordinates:
  * z        (z) MultiIndex
  - x        (z) object 'a' 'a' 'a' 'b' 'b' 'b'
  - y        (z) int64 0 1 2 0 1 2

--------------
unstack z
<xarray.DataArray (x: 2, y: 3)>
array([[ 0.42387457,  1.66837469,  1.00810077],
       [-0.9651798 ,  1.67118143, -0.84389286]])
Coordinates:
  * x        (x) object 'a' 'b'
  * y        (y) int64 0 1 2



## Computation &

### Basic numerical operation

In [11]:
#--- arithmetic operation
xr.set_options(keep_attrs =True)  # keep attributes after operations
temp_degC = temp - 273.15
temp_degC.attrs['units']="C"
temp_degC

#--- mean and others
#    a list: mean, min, max, std, sum, weighted
temp_tavg = temp.mean("time")
temp_tavg

temp_tiavg = temp.mean(["time","lon"])
temp_tiavg

<xarray.DataArray (lat: 3)>
array([282.58855448, 283.41587996, 282.04801957])
Coordinates:
  * lat      (lat) float64 -120.0 -30.0 60.0
Attributes:
    units:      K
    long_name:  Temperature

### Rolling average

https://xarray.pydata.org/en/stable/user-guide/computation.html#rolling-window-operations
https://xarray.pydata.org/en/stable/generated/xarray.DataArray.rolling.html#xarray.DataArray.rolling

In [12]:
  #--- pk values. Output directly from AM4 files
  pk_list = [100, 400, 818.6021, 1378.886, 2091.795, 2983.641, 4121.79, 5579.222, 
    6907.19, 7735.787, 8197.665, 8377.955, 8331.696, 8094.722, 7690.857, 
    7139.018, 6464.803, 5712.357, 4940.054, 4198.604, 3516.633, 2905.199, 
    2366.737, 1899.195, 1497.781, 1156.253, 867.792, 625.5933, 426.2132, 
    264.7661, 145.0665, 60, 15, 0]

  #--- bk values. Output directly from AM4 files
  bk_list = [0, 0, 0, 0, 0, 0, 0, 0, 0.00513, 0.01969, 0.04299, 0.07477, 0.11508, 
    0.16408, 0.22198, 0.28865, 0.36281, 0.44112, 0.51882, 0.59185, 0.6581, 
    0.71694, 0.76843, 0.81293, 0.851, 0.88331, 0.91055, 0.93331, 0.95214, 
    0.9675, 0.97968, 0.98908, 0.99575, 1]  

  #--- make DataArray
  ps = xr.DataArray([102000.], dims=['time'])
  pk = xr.DataArray(pk_list, dims=['plev'])
  bk = xr.DataArray(bk_list, dims=['plev'])

  #--- p at half levels: p = ps*bk + pk. ps is the first term because it would be broadcasted.
  phalf = ps*bk + pk

  #--- compute p at full levels, using .rolling(). NaNs is removed by dropna()
  pfull = phalf.rolling(plev=2, center=True).mean().dropna("plev")
  yhc.printv(phalf, "phalf")
  yhc.printv(pfull, "pfull")

--------------
phalf
<xarray.DataArray (time: 1, plev: 34)>
array([[1.00000000e+02, 4.00000000e+02, 8.18602100e+02, 1.37888600e+03,
        2.09179500e+03, 2.98364100e+03, 4.12179000e+03, 5.57922200e+03,
        7.43045000e+03, 9.74416700e+03, 1.25826450e+04, 1.60044950e+04,
        2.00698560e+04, 2.48308820e+04, 3.03328170e+04, 3.65813180e+04,
        4.34714230e+04, 5.07065970e+04, 5.78596940e+04, 6.45673040e+04,
        7.06428330e+04, 7.60330790e+04, 8.07465970e+04, 8.48180550e+04,
        8.82997810e+04, 9.12538730e+04, 9.37438920e+04, 9.58232133e+04,
        9.75444932e+04, 9.89497661e+04, 1.00072427e+05, 1.00946160e+05,
        1.01581500e+05, 1.02000000e+05]])
Dimensions without coordinates: time, plev

--------------
pfull
<xarray.DataArray (time: 1, plev: 33)>
array([[   250.     ,    609.30105,   1098.74405,   1735.3405 ,
          2537.718  ,   3552.7155 ,   4850.506  ,   6504.836  ,
          8587.3085 ,  11163.406  ,  14293.57   ,  18037.1755 ,
         22450.369  ,  275

### Broadcasting by dimension name

https://docs.xarray.dev/en/stable/user-guide/computation.html#broadcasting-by-dimension-name

In [13]:
a = xr.DataArray([1, 2], [("x", ["a", "b"])])
b = xr.DataArray([-1, -2, -3], [("y", [10, 20, 30])])
c = xr.DataArray(np.arange(6).reshape(3, 2), [b["y"], a["x"]])

#print(a)
#print(b)
#print(c)

#--- With xarray, and their dimensions are expanded automatically:
print(a*b)  # 2x3 array
print(b*a)  # 3x2 array

#--- explicitly broadcast xarray data structures by using the broadcast()
a2, b2 = xr.broadcast(a, b)
print(a2)
print(b2)


<xarray.DataArray (x: 2, y: 3)>
array([[-1, -2, -3],
       [-2, -4, -6]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30
<xarray.DataArray (y: 3, x: 2)>
array([[-1, -2],
       [-2, -4],
       [-3, -6]])
Coordinates:
  * y        (y) int64 10 20 30
  * x        (x) <U1 'a' 'b'
<xarray.DataArray (x: 2, y: 3)>
array([[1, 1, 1],
       [2, 2, 2]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30
<xarray.DataArray (x: 2, y: 3)>
array([[-1, -2, -3],
       [-1, -2, -3]])
Coordinates:
  * y        (y) int64 10 20 30
  * x        (x) <U1 'a' 'b'


### Automatic alignment
https://docs.xarray.dev/en/stable/user-guide/computation.html#automatic-alignment



In [14]:
arr = xr.DataArray(np.arange(3), [("x", range(3))])

#--- only operate available elements
print(arr)
print(arr[:-1])
print(arr + arr[:-1])

<xarray.DataArray (x: 3)>
array([0, 1, 2])
Coordinates:
  * x        (x) int64 0 1 2
<xarray.DataArray (x: 2)>
array([0, 1])
Coordinates:
  * x        (x) int64 0 1
<xarray.DataArray (x: 2)>
array([0, 2])
Coordinates:
  * x        (x) int64 0 1
